<a href="https://colab.research.google.com/github/Bolorooo24/CV-ML_labs/blob/main/Lab_3/fine_tune_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Feb 21 12:39:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install --upgrade --quiet pip
!pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "mn", split="train+validation")
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "mn", split="test")

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 4019
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 1886
    })
})


In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 4019
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 1886
    })
})


In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

### Load WhisperTokenizer

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="Mongolian", task="transcribe")

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="Mongolian", task="transcribe")

In [ ]:
print(common_voice["test"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/4780fe25b504ff180c1ded81477eda5c583d8b999c6fb608d52a45015482319b/mn_test_0/common_voice_mn_18785480.mp3', 'array': array([ 0.00000000e+00, -1.63004159e-13, -2.97881945e-13, ...,
       -2.16025292e-05, -8.97848258e-06,  2.05335273e-05]), 'sampling_rate': 48000}, 'sentence': 'Иймэрхүү хоол идсээр байгаад дэндүү туранхай, сул дорой болсон байлаа.'}


In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/641d5f5f7d9dd3b8c9d470043bea832c11480a3932cbeb83ad636bf473b0acda/mn_train_0/common_voice_mn_18577548.mp3', 'array': array([-2.27373675e-12, -4.54747351e-12, -1.81898940e-12, ...,
        1.00528268e-06,  1.37278403e-06, -6.69388101e-09]), 'sampling_rate': 16000}, 'sentence': 'Өглөө таван үг ч хэлүүлэхгүй барьж идчих гээд байх чинь билээ.'}


In [ ]:
def prepare_dataset(batch):

    audio = batch["audio"]

    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]


    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

Map (num_proc=2):   0%|          | 0/4019 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1886 [00:00<?, ? examples/s]

### Load a Pre-Trained Checkpoint

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")

In [ ]:
model.generation_config.language = "Mongolian"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Define a Data Collator

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:

        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        label_features = [{"input_ids": feature["labels"]} for feature in features]

        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")


        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)


        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

### Evaluation Metrics

In [ ]:
import evaluate

metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -100] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Define the Training Configuration

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-tiny-mn",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

<ipython-input-20-69786f5d74d5>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
processor.save_pretrained(training_args.output_dir)

[]

### Training

In [ ]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.504500,0.820233,72.941241
2000,0.225500,0.792130,68.441459
3000,0.099200,0.853391,67.611402
4000,0.059500,0.891888,68.425076


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 2

TrainOutput(global_step=4000, training_loss=0.4519833379089832, metrics={'train_runtime': 10818.298, 'train_samples_per_second': 5.916, 'train_steps_per_second': 0.37, 'total_flos': 1.5708079650816e+18, 'train_loss': 0.4519833379089832, 'epoch': 15.873015873015873})

In [ ]:
trainer.save_model("whisper-tiny-mn")

In [ ]:
tokenizer.save_pretrained("whisper-tiny-mn")

('whisper-tiny-mn/tokenizer_config.json',
 'whisper-tiny-mn/special_tokens_map.json',
 'whisper-tiny-mn/vocab.json',
 'whisper-tiny-mn/merges.txt',
 'whisper-tiny-mn/normalizer.json',
 'whisper-tiny-mn/added_tokens.json')

In [ ]:
from google.colab import drive

import os
if os.path.exists('/content/drive') and os.listdir('/content/drive'):
  !rm -rf /content/drive/*
  print("Cleared existing files in /content/drive")

drive.mount('/content/drive')
print("Drive mounted successfully!")

Mounted at /content/drive
Drive mounted successfully!


In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/Models/whisper1")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/Models/whisper1")

('/content/drive/MyDrive/Colab Notebooks/Models/whisper1/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Models/whisper1/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Models/whisper1/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/Models/whisper1/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/Models/whisper1/normalizer.json',
 '/content/drive/MyDrive/Colab Notebooks/Models/whisper1/added_tokens.json')

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

processor = WhisperProcessor.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Models/whisper1")

model = WhisperForConditionalGeneration.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Models/whisper1", trust_remote_code=True)

print("Fine-tuned Whisper model loaded successfully!")


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Fine-tuned Whisper model loaded successfully!


In [ ]:

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


In [ ]:
import librosa
import torch


sample = common_voice["test"][0]


audio_array = sample["audio"]["array"]
original_sampling_rate = sample["audio"]["sampling_rate"]
audio_array_16k = librosa.resample(audio_array, orig_sr=original_sampling_rate, target_sr=16000)


input_features = processor(audio_array_16k, sampling_rate=16000, return_tensors="pt")
input_features["attention_mask"] = torch.ones_like(input_features["input_features"])


with torch.no_grad():
    predicted_ids = model.generate(
        input_features["input_features"],
        attention_mask=input_features["attention_mask"]
    )


transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print("Transcription:", transcription)


Transcription: Өнөөх байгаа хүнэх байгаа хүнэх байгаа хүүхэд байгаа хүүхэд байгаа хүйнүү хэлэн хүүхэд байгаа хүүйнүү хэлэн хүүүхэд байгаа.


In [ ]:
audio_array, original_sr = librosa.load("/content/drive/MyDrive/Colab Notebooks/Models/test_dataset/test audio.mp3", sr=None)
print(f"Original Sampling Rate: {original_sr} Hz")


Original Sampling Rate: 48000 Hz


In [ ]:
audio_array_16k = librosa.resample(audio_array, orig_sr=original_sr, target_sr=16000)


In [ ]:

input_features = processor(audio_array_16k, sampling_rate=16000, return_tensors="pt").input_features


with torch.no_grad():
    predicted_ids = model.generate(input_features)


transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
print(" Transcription:", transcription)


 Transcription: Этийн хангай саяны өндөр сайхан нөрөөн нутас хойд зүүйн чимээг болсон ойн хувчийн юм болно.
